In [1]:
!pip install ucimlrepo
!pip install scipy
!pip install scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import GridSearchCV


## 1: Obtener el dataset

In [4]:
# Obtener dataset del csv
X_or = pd.read_csv('PCA_datos.csv')['PCA']
Y_or = pd.read_csv('PCA_datos.csv')['Complains']
X_or

KeyError: 'PCA'

## 2: dividir en conjuntos de entrenamiento y test

In [4]:
x_train, x_test, y_train, y_true = train_test_split(X_or,Y_or, test_size=0.3)

# KNN

## 3: Entrenar y ejecutar el modelo

In [5]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [6]:
y_pred_knn = knn.predict(x_test)

## 4: Evaluar el modelo

In [7]:
confusion_matrix(y_true, y_pred_knn)

array([[777,  19],
       [ 52,   3]])

In [8]:
puntuacion_knn_antigua = accuracy_score(y_true, y_pred_knn)
print(puntuacion_knn_antigua)

0.9165687426556992


In [9]:
puntuacion_knn_balanceada_antigua =balanced_accuracy_score(y_true, y_pred_knn)
print(puntuacion_knn_balanceada_antigua)

0.515338053905893


En este caso, es mejor usar la puntuación balanceada, ya que hay muchos más valores de 0 (sin quejas) que de 1.
Esto lleva al KNN a confundirse y tener muchos falsos negativos a la hora de detectar quejas como se ve en la matriz de confusión.

## 5: Optimizar K

In [10]:
clf = GridSearchCV(KNeighborsClassifier(),scoring='balanced_accuracy',param_grid={'n_neighbors':[2,3,4,5,6,7,8,15,16,31,32]})

clf.fit(X_or,Y_or)
df_resultados_grid = pd.DataFrame(clf.cv_results_)
df_resultados_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002596,0.000394,0.025624,0.002781,2,{'n_neighbors': 2},0.504165,0.499040,0.501273,0.489443,0.506071,0.499998,0.005800,8
1,0.002031,0.000048,0.022355,0.002086,3,{'n_neighbors': 3},0.497460,0.516941,0.502545,0.511495,0.503192,0.506327,0.006959,1
2,0.002255,0.000642,0.021222,0.000311,4,{'n_neighbors': 4},0.505122,0.500000,0.492322,0.500313,0.509910,0.501534,0.005861,3
3,0.001945,0.000055,0.021088,0.000349,5,{'n_neighbors': 5},0.513118,0.498081,0.499353,0.507344,0.509910,0.505561,0.005894,2
4,0.002061,0.000161,0.021027,0.000409,6,{'n_neighbors': 6},0.509912,0.499040,0.495202,0.495202,0.500000,0.499871,0.005388,9
5,0.001966,0.000046,0.021521,0.000542,7,{'n_neighbors': 7},0.509912,0.498081,0.492322,0.491363,0.500000,0.498336,0.006658,11
6,0.002014,0.000170,0.020874,0.000321,8,{'n_neighbors': 8},0.500000,0.500000,0.499040,0.499040,0.500000,0.499616,0.000470,10
7,0.001936,0.000062,0.021774,0.000414,15,{'n_neighbors': 15},0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,4
8,0.001894,0.000052,0.021273,0.000620,16,{'n_neighbors': 16},0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,4
9,0.002095,0.000143,0.023413,0.001583,31,{'n_neighbors': 31},0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,4


In [11]:
clf.best_estimator_

KNeighborsClassifier(n_neighbors=3)

Con estos resultados, queda probado que el mejor estimador entre los posibles k escogidos es k=15

## 6: Reentrenar KNN con el k seleccionado

In [12]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_test)

In [13]:
confusion_matrix(y_true, y_pred_knn)

array([[766,  30],
       [ 49,   6]])

In [14]:
puntuacion_knn_nueva = accuracy_score(y_true, y_pred_knn)

In [15]:
puntuacion_knn_balanceada_nueva = balanced_accuracy_score(y_true, y_pred_knn)

In [16]:
print('La puntuación sin balancear ha pasado de {0} a {1}'.format(puntuacion_knn_antigua, puntuacion_knn_nueva))
print('La puntuación balanceada ha pasado de {0} a {1}'.format(puntuacion_knn_balanceada_antigua, puntuacion_knn_balanceada_nueva))


La puntuación sin balancear ha pasado de 0.9165687426556992 a 0.9071680376028202
La puntuación balanceada ha pasado de 0.515338053905893 a 0.5357012334399269


Como estábamos priorizando la puntuación balanceada, al haber mejora en esta, podemos decir que el modelo ha mejorado.

# Árboles de decisión

In [17]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier


## 3: Entrenar y ejecutar el modelo

In [18]:
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [19]:
y_pred_arbol = clf.predict(x_test)

## 4: Evaluar el modelo

In [20]:
confusion_matrix(y_true, y_pred_arbol)

array([[743,  53],
       [ 48,   7]])

In [21]:
puntuacion_arbol_antigua = accuracy_score(y_true, y_pred_arbol)
print(puntuacion_arbol_antigua)

0.881316098707403


In [22]:
puntuacion_arbol_balanceada_antigua = balanced_accuracy_score(y_true, y_pred_arbol)
print(puntuacion_arbol_balanceada_antigua)

0.5303449063499315


Estamos ante el mismo caso del KNN

## 5: Optimizar el árbol

In [23]:
clf = GridSearchCV(DecisionTreeClassifier(),scoring='balanced_accuracy',param_grid={'max_depth':[2,3,4,5,6,7,8,9,10,15,30,90,270]})

In [24]:
clf.fit(X_or,Y_or)
df_resultados_grid = pd.DataFrame(clf.cv_results_)
df_resultados_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003389,0.000600,0.002468,0.000409,2,{'max_depth': 2},0.500000,0.500000,0.500000,0.510870,0.498081,0.501790,0.004600,4
1,0.003146,0.000102,0.002023,0.000083,3,{'max_depth': 3},0.498084,0.498081,0.500000,0.500000,0.498081,0.498849,0.000940,9
2,0.003646,0.000382,0.002495,0.000400,4,{'max_depth': 4},0.498084,0.497121,0.507990,0.498081,0.498081,0.499871,0.004077,8
3,0.003519,0.000191,0.001894,0.000129,5,{'max_depth': 5},0.498084,0.497121,0.507990,0.499040,0.498081,0.500063,0.004010,6
4,0.003574,0.000087,0.001828,0.000062,6,{'max_depth': 6},0.498084,0.503192,0.507031,0.499040,0.494242,0.500318,0.004401,5
5,0.003788,0.000064,0.001807,0.000070,7,{'max_depth': 7},0.507038,0.503192,0.504152,0.493282,0.492322,0.499997,0.006017,7
6,0.004177,0.000211,0.001926,0.000178,8,{'max_depth': 8},0.505122,0.499353,0.504152,0.490403,0.492322,0.498271,0.006000,10
7,0.005210,0.000847,0.002541,0.000394,9,{'max_depth': 9},0.502249,0.494555,0.500313,0.484645,0.491363,0.494625,0.006336,11
8,0.005051,0.000393,0.002199,0.000169,10,{'max_depth': 10},0.500333,0.494555,0.498394,0.481766,0.489443,0.492898,0.006695,13
9,0.006795,0.000933,0.002643,0.000462,15,{'max_depth': 15},0.520490,0.488797,0.494868,0.466411,0.502545,0.494622,0.017674,12


In [25]:
clf.best_estimator_

DecisionTreeClassifier(max_depth=30)

## 6: Reentrenar el árbol con el max_depth seleccionado

In [26]:
clf = DecisionTreeClassifier(max_depth=30)
clf.fit(x_train, y_train)
y_pred_arbol = clf.predict(x_test)

In [27]:
confusion_matrix(y_true, y_pred_arbol)

array([[743,  53],
       [ 48,   7]])

In [28]:
puntuacion_arbol_nueva = accuracy_score(y_true, y_pred_arbol)

In [29]:
puntuacion_arbol_balanceada_nueva = balanced_accuracy_score(y_true, y_pred_arbol)

In [30]:
print('La puntuación sin balancear ha pasado de {0} a {1}'.format(puntuacion_arbol_antigua, puntuacion_arbol_nueva))
print('La puntuación balanceada ha pasado de {0} a {1}'.format(puntuacion_arbol_balanceada_antigua, puntuacion_arbol_balanceada_nueva))

La puntuación sin balancear ha pasado de 0.881316098707403 a 0.881316098707403
La puntuación balanceada ha pasado de 0.5303449063499315 a 0.5303449063499315


# Comparación de resultados

In [31]:
print('La puntuación sin balancear del KNN ha pasado de {0} a {1}'.format(puntuacion_knn_antigua, puntuacion_knn_nueva))
print('La puntuación balanceada del KNN ha pasado de {0} a {1}'.format(puntuacion_knn_balanceada_antigua, puntuacion_knn_balanceada_nueva))
print('La puntuación sin balancear del árbol ha pasado de {0} a {1}'.format(puntuacion_arbol_antigua, puntuacion_arbol_nueva))
print('La puntuación balanceada del árbol ha pasado de {0} a {1}'.format(puntuacion_arbol_balanceada_antigua, puntuacion_arbol_balanceada_nueva))

La puntuación sin balancear del KNN ha pasado de 0.9165687426556992 a 0.9071680376028202
La puntuación balanceada del KNN ha pasado de 0.515338053905893 a 0.5357012334399269
La puntuación sin balancear del árbol ha pasado de 0.881316098707403 a 0.881316098707403
La puntuación balanceada del árbol ha pasado de 0.5303449063499315 a 0.5303449063499315


En este caso el KNN ha tenido una mejora de resultados más pronunciada y tuvo los mejores resultados